In [1]:
import numpy as np
import pandas as pd
import datetime as dt

import json
from PIL import Image
import urllib.request
import requests
from bs4 import BeautifulSoup

# Background

https://www.irishtimes.com/life-and-style/people/ireland-s-darkest-oddest-and-weirdest-secrets-uncovered-1.3418059

https://www.duchas.ie/download/17.01.26-irish-folklore-and-tradition.pdf

https://www.frontiersin.org/articles/10.3389/fphar.2020.584595/full

# Example
https://www.duchas.ie/en/tpc/cbes/5098227?p=1

# Full pipeline

In [2]:
pp = 1
base_url = 'https://www.duchas.ie'
all_topics = base_url+'/en/tpc/cbes'
print(all_topics)
page_all = requests.get(all_topics)
soup_all = BeautifulSoup(page_all.content, 'html.parser')
all_topics_list = soup_all.find_all('div',{'class':'topics'})[0].find_all('a',href=True)

https://www.duchas.ie/en/tpc/cbes


In [3]:
all_data = {}
t_ix = 0
curr_idx = 0

In [ ]:
curr_idx = curr_idx + t_ix + 1
for t_ix, t_url in enumerate(all_topics_list[curr_idx:curr_idx+40]):
    topic_url = t_url['href']

    topic_url_i = base_url+topic_url
    page_i = requests.get(topic_url_i)
    soup_i = BeautifulSoup(page_i.content, 'html.parser')
    topic_name_i = soup_i.find_all('div',{'class':'pageHeader'})[0].text.replace('\n','')
    n_results_topic_i = int(soup_i.find('div',{'class':'resultCount'}).text.split('count: '
                                            )[1].replace(' ','').replace(',','').split('\r')[0])
    n_pages_topic_i = int(soup_i.find('span',{'class':'lastPageNum'}).text.replace(' ','').replace('/',''))
    print(dt.datetime.now(),'\tStarting',topic_name_i)
    out_topic_dict = {}

#     for pp in range(1,14):
    for pp in range(1,n_pages_topic_i+1):
        topic_url_pp = base_url+topic_url+'?p=%i'%pp
        page_pp = requests.get(topic_url_pp)
        soup_pp = BeautifulSoup(page_pp.content, 'html.parser')
        try:
            all_stories_pp = soup_pp.find_all('section',
                                        {'class':'contentListBox'})[0].find_all('div',
                                                                                {'class':'contentDesc'})
        except:
            print("\t\tbroken link:",topic_url_pp)
            continue

        print('\t',topic_name_i,'page: %03i/%i ... %i results on this page'%(pp,n_pages_topic_i,
                                                        len(all_stories_pp)), dt.datetime.now())
#         for sub_story_i in all_stories_pp[:2]:
        for sub_story_i in all_stories_pp:
            sub_story_url = base_url+sub_story_i.find('a',href=True)['href']
            page_i = requests.get(sub_story_url)
            soup_i = BeautifulSoup(page_i.content, 'html.parser')
            text_i = soup_i.find_all('div',{'class':'transcript'})
            meta_data_list = [dict(zip([i.text for i in x.find_all('dt')],
                                       [i.text for i in x.find_all('dd')]))
                              for x in soup_i.find_all('dl')]

            raw_text_i = "\n\n".join([i.text for i in
                                      soup_i.find_all('div',{'class':'transcript'})])
            try:
                bibl1 = soup_i.find_all('div',{'id':'itemBibli'})[0].find('h2').text
                bibl2 = soup_i.find_all('div',{'id':'itemBibli'})[0].find('p').text

            except:
                print("\t\tbroken link:",topic_url_pp)
                continue

            school_info_i = soup_i.find_all('div',{'class':'xml'})[0].text
            page_info_i = soup_i.find_all('div',{'class':'xml'})[1].text
            title_i = soup_i.find_all('div',{'class':'xml'}
                                     )[2].text.replace('“','').replace('”','')
            label_id = sub_story_url.replace('https://www.duchas.ie/en/cbes/','')
            out_topic_dict[label_id] = {'title':title_i,
                                        'bib':[bibl1,bibl2,page_info_i],
                                        'url':sub_story_url,
                                        'text':raw_text_i,
                                        'school_info':school_info_i,
                                        'metadata':meta_data_list
                                         }

    with open('../data/out/schools-collection_%s.json'%topic_name_i, 'w') as fp:
        json.dump(out_topic_dict, fp)

    print(dt.datetime.now(),'\tSaving....',topic_name_i,t_ix,'\n')

    all_data[topic_name_i] = out_topic_dict

2021-05-11 08:27:56.820844 	Starting Land management
	 Land management page: 001/208 ... 20 results on this page 2021-05-11 08:27:57.745057
	 Land management page: 002/208 ... 20 results on this page 2021-05-11 08:28:10.903889
	 Land management page: 003/208 ... 20 results on this page 2021-05-11 08:28:21.369341
	 Land management page: 004/208 ... 20 results on this page 2021-05-11 08:28:31.376615
	 Land management page: 005/208 ... 20 results on this page 2021-05-11 08:28:43.865839
	 Land management page: 006/208 ... 20 results on this page 2021-05-11 08:28:53.549817
	 Land management page: 007/208 ... 20 results on this page 2021-05-11 08:29:03.898115
	 Land management page: 008/208 ... 20 results on this page 2021-05-11 08:29:13.664030
	 Land management page: 009/208 ... 20 results on this page 2021-05-11 08:29:24.029220
	 Land management page: 010/208 ... 20 results on this page 2021-05-11 08:29:33.422760
	 Land management page: 011/208 ... 20 results on this page 2021-05-11 08:29:

_________________________